In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

from app.modules.audio.AudioData import AudioData
from app.modules.audio.AudioPlayer import AudioPlayer
from app.modules.midi.MidiData import MidiData
from app.modules.midi.MidiPlayer import MidiPlayer
from app.modules.midi.MidiSynth import MidiSynth

from app.modules.pitch.pda.PYin import PYin
from app.modules.pitch.Onsets import Onsets
from app.config import AppConfig
from app.modules.dtw.DTW import CQT, DTW

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [5]:
AUDIO_FILEPATH = '../../app/resources/audio/user_fugue2.mp3'
MIN_VIOLIN_FREQ = 196.0
SAMPLE_RATE = 44100

audio_data = AudioData()
audio_data.load_data(AUDIO_FILEPATH)
pitches, most_likely_pitches = PYin.pyin(audio_data.data)

Processing frame 4604/4604
Done!


In [10]:
# Create a synth with a soundfont
SOUNDFONT_FILEPATH = '../../app/resources/MuseScore_General.sf3'
midi_synth = MidiSynth(SOUNDFONT_FILEPATH)

# Load the midi file into a MidiData object
MIDI_FILEPATH = '../../app/resources/midi/fugue.mid'
midi_data = MidiData(MIDI_FILEPATH)

# Create MidiSynth/Player objects
midi_player = MidiPlayer(midi_synth)
midi_player.load_midi(midi_data)

# midi_player.play(start_time=0) # Play the MIDI

Loading MidiSynth...
Synth + soundfont loaded.


## Sanity check: Plot pitches

In [6]:
# View the pitch estimates in the app
import sys
sys.path.append('..')

from app.ui.plots.PitchPlot import RunPitchPlot
from app.modules.midi.MidiData import MidiData
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication


if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    MIDI_FILEPATH = '../../app/resources/midi/fugue.mid'
    midi_data = MidiData(MIDI_FILEPATH)

    pitchplot = RunPitchPlot(
        app, midi_data=midi_data, pitches=pitches
    )

Plotting pitches...
Done!


### Compute Onset Data
We compute `onset_times` with Essentia's 'complex' onset detection which works better for non-percussive detection (eg, music signals). Analyzes the spectrum of the audio and searches for timbrally-percussive spectra (probably). Catches note changes which are the same pitch.

We compute `note_df` based off different-enough pitch changes using a rolling median window. Tries to catch note changes which may have less percussiveness (eg, slurred notes).

We combine them into `onset_df` to capture as many types of note changes as we can, mergining those onsets within `combine_threshold` sec tolerance into the same row of the dataframe, to be considered the same onset for warping purposes.

In [6]:
os = Onsets()
onset_times = os.detect_onsets(audio_data)
note_df = os.detect_pitch_changes(most_likely_pitches, window_size=15, threshold=0.4)
onset_df = os.combine_onsets(onset_times=onset_times, 
                             note_df=note_df, 
                             combine_threshold=0.01)

# View the combined onset_df based on both onsets and pitch differences
onset_df

Detecting onsets... Done!
Segmenting notes with window_size=15 and threshold=0.4... Done!


,time,pitch_diff,onset
0,0.000000,True,False
1,0.011610,False,True
2,0.232200,False,True
3,0.296054,True,False
4,0.464399,False,True
...,...,...,...
128,12.585216,False,True
129,12.704218,True,False
130,12.759365,False,True
131,12.910295,False,True


In [9]:
cqt_onset_df = DTW.cqt_onset_df(audio_data.data, onset_df)

cqt_onset_df

,time,pitch_diff,onset,cqt_norm
0,0.000000,True,False,"[-0.080838166, -0.0668917, -0.064405546, -0.05..."
1,0.011610,False,True,"[-0.09454665, -0.093443535, -0.0933609, -0.088..."
2,0.232200,False,True,"[-0.081852324, -0.07638682, -0.07203554, -0.06..."
3,0.296054,True,False,"[-0.11152571, -0.10601042, -0.10273066, -0.098..."
4,0.464399,False,True,"[-0.22583568, -0.14463839, -0.12643395, -0.124..."
...,...,...,...,...
128,12.585216,False,True,"[-0.10679642, -0.10286422, -0.10027398, -0.095..."
129,12.704218,True,False,"[-0.12708835, -0.12495376, -0.12868106, -0.132..."
130,12.759365,False,True,"[-0.13078195, -0.11990827, -0.119574085, -0.11..."
131,12.910295,False,True,"[-0.090926185, -0.089150496, -0.08656771, -0.0..."
